### Курсовая работа по теме "Распознавание лиц и эмоций"

Для решения задачи будем использовать библиотеку mediapipe для детектирования лиц и жестов. Сначала модель детектирует лицо пользователя,а потом детектирует и классифицирует показанный им жест (за основу взяты три жеста: сложенная ладонь, ОК, пятерня. Mediapipe детектирует жесты в 2 этапа: - детектирование, а затем обнаружение особых точек (landmarks) жеста. - выдача результата.
Будем работать с (landmarks) жеста и их координатами, а не bounding box-ами детектированных жестов, поэтому в качестве классификатора у нас будет обычная логистическая регрессия, а не свёрточная нейронная сеть.  PS: для выхода и завершения работы web-камеры нажми "ESC"

In [20]:
import numpy as np
import pandas as pd
import re
import cv2
import glob
import mediapipe as mp


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [21]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands

In [22]:
def landmarks_to_df(landmarks, df=None, target=None):
    
    # Переводим в привычный словарь
    landmarks = MessageToDict(landmarks[0])['landmark']
    
    # Заполняем списки с координатами
    x = np.empty(len(landmarks))
    y = np.empty(len(landmarks))
    z = np.empty(len(landmarks))
    
    for i, v in enumerate(landmarks):
        x[i] = v['x']
        y[i] = v['y']
        z[i] = v['z']
    
    # Масшабируем координаты, чтобы не имело значения, в какой
    # области кадра наш жест
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x.reshape(-1, 1)).reshape(1, -1)
    y = scaler.fit_transform(y.reshape(-1, 1)).reshape(1, -1)
    z = scaler.fit_transform(z.reshape(-1, 1)).reshape(1, -1)
    
    # Подготавливаем и возвращаем датасет
    if target is not None:
        features = np.c_[x, y, z, target]
    else:
        features = np.c_[x, y, z]
    
    if df is not None:
        df = pd.concat((df, pd.DataFrame(features)), axis=0)
    else:
        df = pd.DataFrame(features)
    
    return df

In [23]:
import glob

In [24]:
files = glob.glob('./leapGestRecog/*/0[13457]*/*') + glob.glob('./leapGestRecog/*/10*/*')
files

['./leapGestRecog\\00\\01_palm\\frame_00_01_0001.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0002.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0003.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0004.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0005.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0006.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0007.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0008.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0009.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0010.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0011.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0012.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0013.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0014.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0015.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0016.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0017.png',
 './leapGestRecog\\00\\01_palm\\frame_00_01_0018.png',
 './leapGe

In [25]:
from google.protobuf.json_format import MessageToDict

In [26]:
mapping_dict = {'01': 1, '07': 2, '10': 3}
df = pd.DataFrame()

with mp_hands.Hands(max_num_hands=1, static_image_mode=True) as hands:
    
        for idx, file in enumerate(files):
            image = cv2.imread(file)
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if not results.multi_hand_landmarks:
                continue
            
            target = mapping_dict.get(re.findall('frame_\d{2}_(\d{2})', file)[0], 0)
            df = landmarks_to_df(results.multi_hand_landmarks, df, target)

In [27]:
df.shape

(5878, 64)

In [28]:
df[63].value_counts()

63
1.0    1855
2.0    1610
0.0    1244
3.0    1169
Name: count, dtype: int64

In [29]:
df.to_csv('./data.csv', index=False)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, stratify=df[63], random_state=10)

In [33]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [34]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       249
         1.0       1.00      1.00      1.00       371
         2.0       0.99      0.99      0.99       322
         3.0       0.99      1.00      0.99       234

    accuracy                           0.99      1176
   macro avg       0.99      0.99      0.99      1176
weighted avg       0.99      0.99      0.99      1176



In [35]:
model.fit(df.iloc[:, :-1], df.iloc[:, -1]);

In [37]:
# !pip install keyboard

In [38]:
import keyboard

In [40]:

cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection() as face_detection:
    with mp_hands.Hands(max_num_hands=1) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print('Ignoring empty camera frame.')
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)
            
            # Несмотря на наличие параметра min_detection_confidence у FaceDetection(), он,  
            # похоже, не работает. Поэтому зададим порог уверенности вручную (0,8), чтобы  
            # избежать ложных срабатываний детектирования лица
            if results.detections and MessageToDict(results.detections[0])['score'][0] > 0.8:
                
                results = hands.process(image)
                
                if results.multi_hand_landmarks:
                    gesture = model.predict_proba(landmarks_to_df(results.multi_hand_landmarks))[0]
                    
                    if max(gesture) > 0.4:
                        gesture = np.argmax(gesture)
                        
                        match gesture:
                            case 0:
                                text = 'Неизвестный жест!'
                            case 1:
                                text = 'Ладонь!'
                            case 2:
                                text = 'OK!'
                            case 3:
                                text = 'Пятерня!'
                    
                    else:
                        text = 'Это жест?'
            
                
                    cv2.putText(image, text, (170, 50),
                                cv2.FONT_HERSHEY_COMPLEX,
                                1.3, (0, 0, 255), 2)
            
            else:
                cv2.putText(image, 'Лица нет в кадре!', (170, 50),
                            cv2.FONT_HERSHEY_COMPLEX,
                            1.3, (0, 0, 255), 2)
            
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imshow('MediaPipe Hands', image) 
            if cv2.waitKey(5) & 0xFF == 27:
                break

cap.release()
cv2.destroyAllWindows()

if keyboard.is_pressed("Esc"):
        exit()